### Word Group In Youtube

In [230]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [231]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [232]:
# language pair
lang_folder = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 28  # native word end index
unique_word_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 ; 10000 = 85 ; 20000,40000 = 90
include_wordgroup_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 : 10000 = 85 ; 20000,40000 = 90
word_usage_min = 10  # 28,200,1000,5000,10000 = 10 ; 20000 = 7 ; 40000 = 5
word_usage_max = 100

# sentence ratio and time shift
adjust_sent_word_ratio = 15  #28,200 = 15 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
#adjust_text_word_ratio = 5  #28,200 = 5 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
shift = 0.3  # sentence time shift

# func select
if word_end == 28:
    sentence_word_count_number = 2
else:
    sentence_word_count_number = 5   
word_group_adjust_select = False  # True, False; True for selecting word group according to sentence_word_count_number in video_id text analysis
                                  # False for selecting word group according to max word group length 

# shared word frequency
word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

6


In [233]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Data/1-Word Group In Youtube Sentence"

Path(path).mkdir(parents=True, exist_ok=True)

In [234]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by=f"{target_opt_column}", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [235]:
def exract_word_group(text, word_list):
    '''
    exract_word_group(text, word_list): \n
    text is a string sentence, word_list occurs words 
    '''
    words = re.findall(r"\w+", text, re.UNICODE)
    index_list = []

    for i in range(len(words)):
        if words[i] in word_list:
            index_list.append(i)
        else:
            pass

    all_index_list = []
    var_index_list = []
    for j in range(len(index_list)):
        try:
            var1 = index_list[j] + 1  
            var2 = index_list[j+1]
        except:
            var1 = index_list[j] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[j]
            var_index_list.append(var3)
        else:
            var3 = index_list[j]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []

    text_list = []
    for k in max(all_index_list, key=len):  # any error convert k to i
        word = words[k]
        text_list.append(word)
        if len(text_list) >= sentence_word_count_number:
            text = " ".join(text_list)
        else:
            text = f"sentencte_word_count_less_than_{sentence_word_count_number}"
    
    return text 

In [236]:
def df_exract_word_group(df, source_text_column, opt_column, word_list, sent_len=False, sent_len_num=2):
    '''
    df_exract_word_group(df_adjust_text_ratio, sentence, video_id, word_list, sent_len=False, sent_len_num=2): \n
    df_adjust_text_ratio is a dataframe and it includes sentence and video_id columns. \n
    sentence is a string sentence. video_id is optional column value. word_list occurs words \n
    sent_len is sentence length condition. sent_len_num is sentence occurs how many word at least. 
    '''
    df.reset_index(drop=True, inplace=True)
    df_new = pd.DataFrame()
    for i in range(len(df)):
        source_text = df.loc[i,f"{source_text_column}"]
        opt_var = df.loc[i,f"{opt_column}"]
        words = re.findall(r"\w+", source_text, re.UNICODE)
        
        index_list = []
        for j in range(len(words)):
            if words[j] in word_list:
                index_list.append(j)
            else:
                pass

        all_index_list = []
        var_index_list = []
        for k in range(len(index_list)):
            try:
                var1 = index_list[k] + 1  
                var2 = index_list[k+1]
            except:
                var1 = index_list[k] + 1  
                var2 = index_list[-1]
            if var1 == var2:
                var3 = index_list[k]
                var_index_list.append(var3)
            else:
                var3 = index_list[k]
                var_index_list.append(var3)
                var4 = list(var_index_list)
                all_index_list.append(var4)
                var_index_list = []

        text_all_list = []
        for m in all_index_list:
            text_list = [] 
            for n in m:
                word = words[n]
                text_list.append(word)
                if sent_len:
                    if len(text_list) >= sent_len_num:
                        text = " ".join(text_list)
                        text_all_list.append(text)
                    else:
                        pass
                else:
                   text = " ".join(text_list)
                   text_all_list.append(text) 
            
        for search_string in text_all_list:
            #df_var["index"] = i
            #df_var["search_string"] = search_string
            #df_var["sentence"] = source_text
            #df_var["video_id"] = opt_var
            df_list = []
            df_list.append([i,search_string,source_text,opt_var])
            df_var = pd.DataFrame(df_list,columns=["index","search_string",f"{source_text_column}",f"{opt_column}"])
            df_new = pd.concat([df_new, df_var], axis=0)

    return df_new 

In [237]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [238]:
def repetation_ratio(text):
    '''
    repetation_ratio(text) 
    text is a like as sentence and def compute repetation word ratio\n
    repetation_ratio("yok bir şey yok bir şey yok bir şey") 
    '''
    word_list = word_tokenize(text)
    word_num = len(word_list)
    word_unique_num = len(set(word_list))
    ratio = int((word_unique_num/word_num)*100)
    
    return ratio

In [239]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,you,102069964
1,i,94447074
2,the,77481215
3,to,58281119
4,is,50852895
...,...,...
552149,fruitcocktail,6
552150,andthesunlightshining,6
552151,upravo,6
552152,yagawa,6


In [240]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [241]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,you,102069964
1,i,94447074
2,the,77481215
3,to,58281119
4,is,50852895
5,a,49726799
6,not,48510772
7,it,47050281
8,that,35204210
9,and,34806253


In [242]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [243]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [244]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

27

#### Youtube Sentence Analysis

In [245]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:29.416,00:00:33.605,jingle bell jingle bell jingle all the way,tPffr9WPl9Y
1,00:00:33.810,00:00:36.876,shivering cold and snow,tPffr9WPl9Y
2,00:00:37.150,00:00:39.000,what else do you wish for in this festive season,tPffr9WPl9Y
3,00:00:39.257,00:00:43.437,the city of edinburgh has decorated herself wi...,tPffr9WPl9Y
4,00:00:43.796,00:00:50.094,so today we will explore different places in e...,tPffr9WPl9Y
...,...,...,...,...
92058,00:04:58.800,00:05:04.880,or stop your server instances this ensures you...,7Ey_ABmbkok
92059,00:05:04.880,00:05:11.440,you can also create alerts that notify cloud s...,7Ey_ABmbkok
92060,00:05:11.440,00:05:16.800,in mind before moving to the cloud the good ne...,7Ey_ABmbkok
92061,00:05:16.800,00:05:22.640,best practices and proven techniques that can ...,7Ey_ABmbkok


In [246]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [247]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,29.416,33.605,jingle bell jingle bell jingle all the way,tPffr9WPl9Y
1,1,33.810,36.876,shivering cold and snow,tPffr9WPl9Y
2,2,37.150,39.000,what else do you wish for in this festive season,tPffr9WPl9Y
3,3,39.257,43.437,the city of edinburgh has decorated herself wi...,tPffr9WPl9Y
4,4,43.796,50.094,so today we will explore different places in e...,tPffr9WPl9Y
...,...,...,...,...,...
92058,92058,298.800,304.880,or stop your server instances this ensures you...,7Ey_ABmbkok
92059,92059,304.880,311.440,you can also create alerts that notify cloud s...,7Ey_ABmbkok
92060,92060,311.440,316.800,in mind before moving to the cloud the good ne...,7Ey_ABmbkok
92061,92061,316.800,322.640,best practices and proven techniques that can ...,7Ey_ABmbkok


In [248]:
#df_youtube_sentence[df_youtube_sentence["index"] == 1287503]

In [249]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/(len(sent_word)+0.001))*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [250]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id","word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,29.416,33.605,jingle bell jingle bell jingle all the way,tPffr9WPl9Y,12.5,"all, bell, jingle, way",the
1,1,33.810,36.876,shivering cold and snow,tPffr9WPl9Y,25.0,"shivering, cold, snow",and
2,2,37.150,39.000,what else do you wish for in this festive season,tPffr9WPl9Y,60.0,"season, else, festive, wish","what, do, for, you, this, in"
3,3,39.257,43.437,the city of edinburgh has decorated herself wi...,tPffr9WPl9Y,21.4,"city, herself, has, christmas, with, decorated...","of, and, the"
4,4,43.796,50.094,so today we will explore different places in e...,tPffr9WPl9Y,29.4,"explore, here, being, christmas, how, today, p...","will, we, in, is, and"
...,...,...,...,...,...,...,...,...
92058,92058,298.800,304.880,or stop your server instances this ensures you...,7Ey_ABmbkok,20.0,"server, using, paying, instances, stop, or, en...","your, this, not"
92059,92059,304.880,311.440,you can also create alerts that notify cloud s...,7Ey_ABmbkok,18.7,"create, keep, risks, also, cloud, can, spendin...","to, you, that"
92060,92060,311.440,316.800,in mind before moving to the cloud the good ne...,7Ey_ABmbkok,29.4,"are, before, clouds, moving, cloud, mind, good...","to, of, the, in, is"
92061,92061,316.800,322.640,best practices and proven techniques that can ...,7Ey_ABmbkok,21.4,"mitigate, risks, techniques, proven, see, best...","of, and, that"


In [251]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,29.416,33.605,jingle bell jingle bell jingle all the way,tPffr9WPl9Y,12.5,"all, bell, jingle, way",the
1,1,33.810,36.876,shivering cold and snow,tPffr9WPl9Y,25.0,"shivering, cold, snow",and
2,2,37.150,39.000,what else do you wish for in this festive season,tPffr9WPl9Y,60.0,"season, else, festive, wish","what, do, for, you, this, in"
3,3,39.257,43.437,the city of edinburgh has decorated herself wi...,tPffr9WPl9Y,21.4,"city, herself, has, christmas, with, decorated...","of, and, the"
4,4,43.796,50.094,so today we will explore different places in e...,tPffr9WPl9Y,29.4,"explore, here, being, christmas, how, today, p...","will, we, in, is, and"
...,...,...,...,...,...,...,...,...
92058,92058,298.800,304.880,or stop your server instances this ensures you...,7Ey_ABmbkok,20.0,"server, using, paying, instances, stop, or, en...","your, this, not"
92059,92059,304.880,311.440,you can also create alerts that notify cloud s...,7Ey_ABmbkok,18.7,"create, keep, risks, also, cloud, can, spendin...","to, you, that"
92060,92060,311.440,316.800,in mind before moving to the cloud the good ne...,7Ey_ABmbkok,29.4,"are, before, clouds, moving, cloud, mind, good...","to, of, the, in, is"
92061,92061,316.800,322.640,best practices and proven techniques that can ...,7Ey_ABmbkok,21.4,"mitigate, risks, techniques, proven, see, best...","of, and, that"


In [252]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result["word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,1,33.810,36.876,shivering cold and snow,tPffr9WPl9Y,25.0,"shivering, cold, snow",and
1,2,37.150,39.000,what else do you wish for in this festive season,tPffr9WPl9Y,60.0,"season, else, festive, wish","what, do, for, you, this, in"
2,3,39.257,43.437,the city of edinburgh has decorated herself wi...,tPffr9WPl9Y,21.4,"city, herself, has, christmas, with, decorated...","of, and, the"
3,4,43.796,50.094,so today we will explore different places in e...,tPffr9WPl9Y,29.4,"explore, here, being, christmas, how, today, p...","will, we, in, is, and"
4,5,50.504,51.928,so lets go with me,tPffr9WPl9Y,20.0,"so, lets, go, with",me
...,...,...,...,...,...,...,...,...
68775,92057,292.720,298.800,for resources that are not being used along th...,7Ey_ABmbkok,31.2,"being, along, are, resources, lines, start, pr...","to, that, for, not, the"
68776,92058,298.800,304.880,or stop your server instances this ensures you...,7Ey_ABmbkok,20.0,"server, using, paying, instances, stop, or, en...","your, this, not"
68777,92060,311.440,316.800,in mind before moving to the cloud the good ne...,7Ey_ABmbkok,29.4,"are, before, clouds, moving, cloud, mind, good...","to, of, the, in, is"
68778,92061,316.800,322.640,best practices and proven techniques that can ...,7Ey_ABmbkok,21.4,"mitigate, risks, techniques, proven, see, best...","of, and, that"


In [253]:
df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_sentence_ratio = df_adjust_sentence_ratio[~(df_adjust_sentence_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
#df_adjust_sentence_ratio.reset_index(drop=True, inplace=True)
df_adjust_sentence_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word,search_string
1,2,37.150,39.000,what else do you wish for in this festive season,tPffr9WPl9Y,60.0,"season, else, festive, wish","what, do, for, you, this, in",for in this
3,4,43.796,50.094,so today we will explore different places in e...,tPffr9WPl9Y,29.4,"explore, here, being, christmas, how, today, p...","will, we, in, is, and",we will
7,8,66.866,70.225,so there is no point of sitting in home in thi...,tPffr9WPl9Y,35.7,"year, home, sitting, so, time, there, point","no, of, this, in, is",is no
14,21,201.573,208.214,last but not the least we will now visit the m...,tPffr9WPl9Y,26.3,"market, now, christmas, place, ie, interesting...","will, we, not, of, the",not the
15,22,209.111,210.128,it is extremely crowded,tPffr9WPl9Y,50.0,"extremely, crowded","it, is",it is
...,...,...,...,...,...,...,...,...,...
68772,92053,267.200,273.840,options and plans until you find what works be...,7Ey_ABmbkok,31.6,"options, are, things, several, until, best, ca...","what, do, that, for, you, and",for you
68773,92054,273.840,280.400,to reduce your cloud costs for example try not...,7Ey_ABmbkok,29.4,"costs, resources, think, if, cloud, over, prov...","to, not, for, your, you",not to
68774,92056,286.880,292.720,threshold limits and dont forget to auto scale...,7Ey_ABmbkok,23.5,"paying, as, down, auto, otherwise, threshold, ...","to, will, and, you",you will
68777,92060,311.440,316.800,in mind before moving to the cloud the good ne...,7Ey_ABmbkok,29.4,"are, before, clouds, moving, cloud, mind, good...","to, of, the, in, is",to the


In [254]:
df_word_group_time_loc_in_sent_result = word_group_time_loc(df_adjust_sentence_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,for in this,37.959375,38.460417,what else do you wish for in this festive season,tPffr9WPl9Y
1,we will,44.262519,44.787352,so today we will explore different places in e...,tPffr9WPl9Y
2,is no,67.313867,67.705750,so there is no point of sitting in home in thi...,tPffr9WPl9Y
3,not the,202.055982,202.599336,last but not the least we will now visit the m...,tPffr9WPl9Y
4,it is,209.111000,209.376304,it is extremely crowded,tPffr9WPl9Y
...,...,...,...,...,...
42003,for you,270.485773,271.101856,options and plans until you find what works be...,7Ey_ABmbkok
42004,not to,276.802581,277.366882,to reduce your cloud costs for example try not...,7Ey_ABmbkok
42005,you will,291.260000,291.894783,threshold limits and dont forget to auto scale...,7Ey_ABmbkok
42006,to the,312.733793,313.226667,in mind before moving to the cloud the good ne...,7Ey_ABmbkok


In [255]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,for in this,37.659375,38.760417,what else do you wish for in this festive season,tPffr9WPl9Y
1,we will,43.962519,45.087352,so today we will explore different places in e...,tPffr9WPl9Y
2,is no,67.013867,68.005750,so there is no point of sitting in home in thi...,tPffr9WPl9Y
3,not the,201.755982,202.899336,last but not the least we will now visit the m...,tPffr9WPl9Y
4,it is,208.811000,209.676304,it is extremely crowded,tPffr9WPl9Y
...,...,...,...,...,...
42003,for you,270.185773,271.401856,options and plans until you find what works be...,7Ey_ABmbkok
42004,not to,276.502581,277.666882,to reduce your cloud costs for example try not...,7Ey_ABmbkok
42005,you will,290.960000,292.194783,threshold limits and dont forget to auto scale...,7Ey_ABmbkok
42006,to the,312.433793,313.526667,in mind before moving to the cloud the good ne...,7Ey_ABmbkok


In [256]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,for in this,38,39,what else do you wish for in this festive season,tPffr9WPl9Y
1,we will,44,45,so today we will explore different places in e...,tPffr9WPl9Y
2,is no,67,68,so there is no point of sitting in home in thi...,tPffr9WPl9Y
3,not the,202,203,last but not the least we will now visit the m...,tPffr9WPl9Y
4,it is,209,210,it is extremely crowded,tPffr9WPl9Y
...,...,...,...,...,...
42003,for you,270,271,options and plans until you find what works be...,7Ey_ABmbkok
42004,not to,277,278,to reduce your cloud costs for example try not...,7Ey_ABmbkok
42005,you will,291,292,threshold limits and dont forget to auto scale...,7Ey_ABmbkok
42006,to the,312,314,in mind before moving to the cloud the good ne...,7Ey_ABmbkok


In [257]:
df_word_group_time_loc_in_sent_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_sent_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,of that what do we have to do i,32,35,because of that what do we have to do i hear y...,uQXz3m0MFtM
1,for you and that you and your,14,17,and new year whatever that looked like for you...,vCAFrr3EcHg
2,is it no no what what is your,82,86,are you having a nice day thank you the web is...,EM7g24rf8aY
3,on this is your this is your,800,802,want come on this is your this is your brownie...,OA3KUb_WG0U
4,to do that and for me i was,178,180,its colour correction and you know people spen...,9fZhGcIlrzM
...,...,...,...,...,...
42003,i i,5705,5706,like no actually i i like that idea is there a...,sim3KsItki0
42004,a a,411,412,so im going to create like a a half cut crease...,qopdneyZVd4
42005,a a,606,608,of like a a teal blue really really pretty and...,qopdneyZVd4
42006,i i,4284,4285,can i take um i i use my action to look the si...,sim3KsItki0


In [258]:
df_word_group_time_loc_in_sent_result["unique_word_ratio"] = df_word_group_time_loc_in_sent_result["search_string"].apply(lambda text: repetation_ratio(text))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id,unique_word_ratio
0,of that what do we have to do i,32,35,because of that what do we have to do i hear y...,uQXz3m0MFtM,88
1,for you and that you and your,14,17,and new year whatever that looked like for you...,vCAFrr3EcHg,71
2,is it no no what what is your,82,86,are you having a nice day thank you the web is...,EM7g24rf8aY,62
3,on this is your this is your,800,802,want come on this is your this is your brownie...,OA3KUb_WG0U,57
4,to do that and for me i was,178,180,its colour correction and you know people spen...,9fZhGcIlrzM,100
...,...,...,...,...,...,...
42003,i i,5705,5706,like no actually i i like that idea is there a...,sim3KsItki0,50
42004,a a,411,412,so im going to create like a a half cut crease...,qopdneyZVd4,50
42005,a a,606,608,of like a a teal blue really really pretty and...,qopdneyZVd4,50
42006,i i,4284,4285,can i take um i i use my action to look the si...,sim3KsItki0,50


In [259]:
df_word_group_time_loc_in_sent_result_repeat_ratio = df_word_group_time_loc_in_sent_result[df_word_group_time_loc_in_sent_result["unique_word_ratio"] > unique_word_ratio]
df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,search_string,start_time,end_time,sentence,video_id
0,of that what do we have to do i,32,35,because of that what do we have to do i hear y...,uQXz3m0MFtM
1,for you and that you and your,14,17,and new year whatever that looked like for you...,vCAFrr3EcHg
2,is it no no what what is your,82,86,are you having a nice day thank you the web is...,EM7g24rf8aY
3,on this is your this is your,800,802,want come on this is your this is your brownie...,OA3KUb_WG0U
4,to do that and for me i was,178,180,its colour correction and you know people spen...,9fZhGcIlrzM
...,...,...,...,...,...
37055,in a,53,54,easy to clean and disinfect in a commercial di...,R1Dz8OrGAg8
37056,am i,218,220,so am i,Hka9kFuhr1U
37057,is a,805,806,first time because there is a lot of stuff the...,W584Ene0PbE
37058,a i,995,996,im going to change my lip color because a i do...,qopdneyZVd4


In [260]:
df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-260-e137816795b3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)


,search_string,start_time,end_time,sentence,video_id,include_string_ratio
0,of that what do we have to do i,32,35,because of that what do we have to do i hear y...,uQXz3m0MFtM,34
1,for you and that you and your,14,17,and new year whatever that looked like for you...,vCAFrr3EcHg,30
2,is it no no what what is your,82,86,are you having a nice day thank you the web is...,EM7g24rf8aY,28
3,on this is your this is your,800,802,want come on this is your this is your brownie...,OA3KUb_WG0U,49
4,to do that and for me i was,178,180,its colour correction and you know people spen...,9fZhGcIlrzM,28
...,...,...,...,...,...,...
37055,in a,53,54,easy to clean and disinfect in a commercial di...,R1Dz8OrGAg8,7
37056,am i,218,220,so am i,Hka9kFuhr1U,57
37057,is a,805,806,first time because there is a lot of stuff the...,W584Ene0PbE,4
37058,a i,995,996,im going to change my lip color because a i do...,qopdneyZVd4,3


In [261]:
df_word_group_time_loc_include_string_ratio = df_word_group_time_loc_in_sent_result_repeat_ratio[df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] > include_wordgroup_ratio]
df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)
df_word_group_time_loc_include_string_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_include_string_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,search_string,start_time,end_time,sentence,video_id
0,for what it is that we do,1323,1325,for what it is that we do,QoTjKrw2VUI
1,what it is this is the,74,76,so do you know what it is this is the honda,H6E6N70jYqI
2,i will do this this is,140,143,i will do this this is going to happen,Fc80L8NNBgQ
3,do this i will do this,160,163,i must do this i will do this,Fc80L8NNBgQ
4,this on and i have a,397,400,leave this on and i have a gun or not a,M5hEQRvIYBI
...,...,...,...,...,...
476,in a,261,266,in a,wrckuGrLSWE
477,on a,1543,1547,on a,ck7l1mEX9Zs
478,of a,1100,1103,of a,ck7l1mEX9Zs
479,i am,25,27,i am,bx6nqhKmlTI


In [262]:
df_youtube_word_usage = word_usage_result(word_list, df_word_group_time_loc_include_string_ratio, "search_string", "video_id", word_usage_min, word_usage_max)
df_youtube_word_usage.drop_duplicates(inplace=True)
df_youtube_word_usage

,search_string,video_id
0,what i am,zeF54RsdAA0
1,for this,zVpBlu6FUDM
2,have the,yWLrmTkxUG4
3,no no he,y30jmDB7QLc
4,i have no,y30jmDB7QLc
...,...,...
414,that was a,0BsyLPv4EQc
415,do you have a,0BsyLPv4EQc
417,and you,0BsyLPv4EQc
418,this is my,0BsyLPv4EQc


In [263]:
df_youtube_word_usage_merge = pd.merge(df_word_group_time_loc_include_string_ratio,df_youtube_word_usage,how="inner",on=["search_string","video_id"])
df_youtube_word_usage_merge.drop_duplicates(subset=["search_string","video_id"],inplace=True)
df_youtube_word_usage_merge.reset_index(drop=True, inplace=True)
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id
0,for what it is that we do,1323,1325,for what it is that we do,QoTjKrw2VUI
1,what it is this is the,74,76,so do you know what it is this is the honda,H6E6N70jYqI
2,i will do this this is,140,143,i will do this this is going to happen,Fc80L8NNBgQ
3,do this i will do this,160,163,i must do this i will do this,Fc80L8NNBgQ
4,this on and i have a,397,400,leave this on and i have a gun or not a,M5hEQRvIYBI
...,...,...,...,...,...
377,in a,261,266,in a,wrckuGrLSWE
378,on a,1543,1547,on a,ck7l1mEX9Zs
379,of a,1100,1103,of a,ck7l1mEX9Zs
380,i am,25,27,i am,bx6nqhKmlTI


In [264]:
df_youtube_word_usage_merge["video_url"] = "https://www.youtube.com/watch?v="+df_youtube_word_usage_merge['video_id'].map(str)+"&t="+df_youtube_word_usage_merge['start_time'].map(str)+"s"
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id,video_url
0,for what it is that we do,1323,1325,for what it is that we do,QoTjKrw2VUI,https://www.youtube.com/watch?v=QoTjKrw2VUI&t=...
1,what it is this is the,74,76,so do you know what it is this is the honda,H6E6N70jYqI,https://www.youtube.com/watch?v=H6E6N70jYqI&t=74s
2,i will do this this is,140,143,i will do this this is going to happen,Fc80L8NNBgQ,https://www.youtube.com/watch?v=Fc80L8NNBgQ&t=...
3,do this i will do this,160,163,i must do this i will do this,Fc80L8NNBgQ,https://www.youtube.com/watch?v=Fc80L8NNBgQ&t=...
4,this on and i have a,397,400,leave this on and i have a gun or not a,M5hEQRvIYBI,https://www.youtube.com/watch?v=M5hEQRvIYBI&t=...
...,...,...,...,...,...,...
377,in a,261,266,in a,wrckuGrLSWE,https://www.youtube.com/watch?v=wrckuGrLSWE&t=...
378,on a,1543,1547,on a,ck7l1mEX9Zs,https://www.youtube.com/watch?v=ck7l1mEX9Zs&t=...
379,of a,1100,1103,of a,ck7l1mEX9Zs,https://www.youtube.com/watch?v=ck7l1mEX9Zs&t=...
380,i am,25,27,i am,bx6nqhKmlTI,https://www.youtube.com/watch?v=bx6nqhKmlTI&t=25s


In [265]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(2000)
df_youtube_word_usage_merge.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.csv", index=False)

In [266]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(700)
df_youtube_word_usage_merge.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

In [267]:
#df_word_group_time_loc_in_sent_result = df_word_group_time_loc_in_sent_result.head(1000000)
#df_word_group_time_loc_in_sent_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [268]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.*")
output_file

['English_Word_Group_Max_In_Youtube_Sentence_40000_Word6.csv',
 'English_Word_Group_Max_In_Youtube_Sentence_40000_Word6.xlsx',
 'English_Word_Group_Max_In_Youtube_Sentence_20000_Word6.csv',
 'English_Word_Group_Max_In_Youtube_Sentence_20000_Word6.xlsx',
 'English_Word_Group_Max_In_Youtube_Sentence_10000_Word6.csv',
 'English_Word_Group_Max_In_Youtube_Sentence_10000_Word6.xlsx',
 'English_Word_Group_Max_In_Youtube_Sentence_5000_Word6.csv',
 'English_Word_Group_Max_In_Youtube_Sentence_5000_Word6.xlsx',
 'English_Word_Group_Max_In_Youtube_Sentence_1000_Word6.csv',
 'English_Word_Group_Max_In_Youtube_Sentence_1000_Word6.xlsx',
 'English_Word_Group_Max_In_Youtube_Sentence_200_Word6.csv',
 'English_Word_Group_Max_In_Youtube_Sentence_200_Word6.xlsx',
 'English_Word_Group_Max_In_Youtube_Sentence_28_Word6.csv',
 'English_Word_Group_Max_In_Youtube_Sentence_28_Word6.xlsx']

In [269]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [270]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass